In [1]:
import pandas as pd
import numpy as np
from itertools import combinations, product

In [2]:
drivers = pd.read_csv('f1 - drivers.csv')

In [3]:
constructors = pd.read_csv('f1 - constructors.csv')

In [4]:
drivers.head()

,Driver,Constructor,Odds,Implied Win Probability,Season Points,Season Points per Race,Cost
0,Lando Norris,McLaren,220,0.312500,569,29.9,30.8
1,Charles Leclerc,Ferrari,1600,0.058824,269,14.2,22.9
2,Max Verstappen,Red Bull,210,0.322581,482,25.4,28.8
3,Oscar Piastri,McLaren,250,0.285714,562,29.6,26.6
4,Lewis Hamilton,Ferrari,2500,0.038462,300,15.8,22.8


In [5]:
# points per position: how many points does each race position score, on average?
ppp = drivers.sort_values('Season Points per Race',ascending=False)['Season Points per Race']

In [6]:
# expected finish order
efo = drivers.sort_values(['Implied Win Probability','Season Points per Race'],ascending=False)[['Driver','Season Points per Race','Cost','Constructor']]

In [7]:
# expected points (weighting is untested)
efo['Avg. Points per Position'] = list(ppp)
efo['Expected Points'] = 0.75*efo['Avg. Points per Position'] + 0.25*efo['Season Points per Race']
drivers = efo

In [8]:
# list every combination of 5 drivers 'dri'
dri = []
for driver_combo in combinations(drivers['Driver'].values,5):
    dri.append(driver_combo)

In [9]:
# list every combination of 2 constructors 'con'
con = []
for con_combo in combinations(constructors['Team'].values,2):
    con.append(con_combo)

In [10]:
# list every combination of those dri and con 'lineup'
lineup = []
for lineup_product in product(dri,con):
    lineup.append(lineup_product)

In [11]:
# pull the lineups into a dataframe
rows = []
for entry in lineup:
    d1 = entry[0][0]
    d2 = entry[0][1]
    d3 = entry[0][2]
    d4 = entry[0][3]
    d5 = entry[0][4]
    c1 = entry[1][0]
    c2 = entry[1][1]
    row = [d1,d2,d3,d4,d5,c1,c2]
    rows.append(row)
cols = ['D1','D2','D3','D4','D5','C1','C2']
df = pd.DataFrame(rows,columns=cols)

In [12]:
# map costs back onto dataframe
driver_map = drivers[['Driver','Cost']].set_index('Driver')['Cost'].to_dict()

for col in ['D1','D2','D3','D4','D5']:
    new_col_str = col + '_budget'
    df[new_col_str] = df[col].map(driver_map)

con_map = constructors[['Team','Cost']].set_index('Team')['Cost'].to_dict()

for col in ['C1','C2']:
    new_col_str = col + '_budget'
    df[new_col_str] = df[col].map(con_map)

df['total_budget'] = df[['D1_budget','D2_budget','D3_budget','D4_budget','D5_budget','C1_budget','C2_budget']].sum(axis=1)

In [13]:
# sum constructor expected points from its drivers
constructors['Expected Points'] = constructors['Team'].map(drivers.groupby('Constructor')['Expected Points'].sum())

In [14]:
# map win probabilities back onto dataframe
driver_map = drivers[['Driver','Expected Points']].set_index('Driver')['Expected Points'].to_dict()

for col in ['D1','D2','D3','D4','D5']:
    new_col_str = col + '_xp'
    df[new_col_str] = df[col].map(driver_map)

con_map = constructors[['Team','Expected Points']].set_index('Team')['Expected Points'].to_dict()

for col in ['C1','C2']:
    new_col_str = col + '_xp'
    df[new_col_str] = df[col].map(con_map)

# the top driver you assign your 'DRS Boost' to gets their score doubled
df['D1_xp'] = df['D1_xp']*2

df['total_xp'] = df[['D1_xp','D2_xp','D3_xp','D4_xp','D5_xp','C1_xp','C2_xp']].sum(axis=1)

In [15]:
# reset index
df = df.reset_index()

In [16]:
# list the remaining lineups by combined expected points
drop = ['D1_xp', 'D2_xp', 'D3_xp', 'D4_xp',
       'D5_xp', 'C1_xp', 'C2_xp','D1_budget',
       'D2_budget', 'D3_budget', 'D4_budget', 'D5_budget', 'C1_budget',
       'C2_budget']
df.drop(columns=drop,inplace=True)

In [17]:
# sort by winningest
df.sort_values(['total_xp','total_budget'],ascending=[False,True],inplace=True)

In [18]:
def best_available_lineups(df,team:list,budget:float,free_transfers:int):
    """
    given a dataframe of upcoming race probs and your team info, spits out the best lineups to switch to
    df: pandas dataframe of all possible lineups, with corresponding total_budget and total_wp
    team_n: a list of the team's current entrants
    budget: the team's current cost cap
    free_transfers: the number of free transfers available
    """
    df = df[df['total_budget'] <= budget] # filter by budget

    # subset to teams that share enough entrants with the current entry
    # to stay under the free transfer limit
    t1 = set(team)
    match_indices = [] # empty list to store match indices
    for possible_entry in df.values:
        t2 = set(possible_entry)
        if len(t1&t2) >= (7 - free_transfers):
            match_indices.append(possible_entry[0])
    df = df[df['index'].isin(match_indices)]
    return df

In [19]:
# team 1
team_1 = ['Oscar Piastri','Liam Lawson','Isack Hadjar','Fernando Alonso','Carlos Sainz','McLaren','Williams']
t1_new = best_available_lineups(df,team_1,budget=110.3,free_transfers=2)
t1_new.head()

,index,D1,D2,D3,D4,D5,C1,C2,total_budget,total_xp
152598,152598,Max Verstappen,Isack Hadjar,Carlos Sainz,Fernando Alonso,Oliver Bearman,McLaren,Williams,109.3,150.925
152553,152553,Max Verstappen,Isack Hadjar,Carlos Sainz,Fernando Alonso,Liam Lawson,McLaren,Williams,108.7,150.625
152643,152643,Max Verstappen,Isack Hadjar,Carlos Sainz,Fernando Alonso,Nico Hulkenberg,McLaren,Williams,110.2,150.475
152913,152913,Max Verstappen,Isack Hadjar,Carlos Sainz,Liam Lawson,Oliver Bearman,McLaren,Williams,109.1,149.925
290703,290703,Lando Norris,Isack Hadjar,Carlos Sainz,Liam Lawson,Gabriel Bortoleto,McLaren,Williams,110.1,149.625


In [20]:
# team 2
team_2 = ['Oscar Piastri','Fernando Alonso','Pierre Gasly','Gabriel Bortoleto','Carlos Sainz','McLaren','Ferrari']
t2_new = best_available_lineups(df,team_2,budget=10000,free_transfers=10)
t2_new.head()

,index,D1,D2,D3,D4,D5,C1,C2,total_budget,total_xp
1,1,Max Verstappen,Lando Norris,Oscar Piastri,George Russell,Charles Leclerc,McLaren,Red Bull,197.1,242.975
46,46,Max Verstappen,Lando Norris,Oscar Piastri,George Russell,Lewis Hamilton,McLaren,Red Bull,197.0,242.175
2,2,Max Verstappen,Lando Norris,Oscar Piastri,George Russell,Charles Leclerc,McLaren,Mercedes,194.4,239.300
47,47,Max Verstappen,Lando Norris,Oscar Piastri,George Russell,Lewis Hamilton,McLaren,Mercedes,194.3,238.500
0,0,Max Verstappen,Lando Norris,Oscar Piastri,George Russell,Charles Leclerc,McLaren,Ferrari,198.4,237.800


In [21]:
# team 3
team_3 = ['Oscar Piastri','Isack Hadjar','Lance Stroll','Fernando Alonso','Liam Lawson','McLaren','Racing Bulls']
t3_new = best_available_lineups(df,team_3,budget=109.5,free_transfers=2)
t3_new.head()

,index,D1,D2,D3,D4,D5,C1,C2,total_budget,total_xp
290256,290256,Lando Norris,Isack Hadjar,Carlos Sainz,Fernando Alonso,Liam Lawson,McLaren,Racing Bulls,106.4,149.575
293496,293496,Lando Norris,Isack Hadjar,Fernando Alonso,Liam Lawson,Oliver Bearman,McLaren,Racing Bulls,107.2,148.275
290481,290481,Lando Norris,Isack Hadjar,Carlos Sainz,Fernando Alonso,Lance Stroll,McLaren,Racing Bulls,109.2,148.175
154176,154176,Max Verstappen,Isack Hadjar,Yuki Tsunoda,Fernando Alonso,Liam Lawson,McLaren,Racing Bulls,107.7,147.975
293541,293541,Lando Norris,Isack Hadjar,Fernando Alonso,Liam Lawson,Nico Hulkenberg,McLaren,Racing Bulls,108.1,147.825


In [22]:
drivers['Points Value'] = drivers['Expected Points']/drivers['Cost']
drivers.sort_values('Points Value',ascending=False)

,Driver,Season Points per Race,Cost,Constructor,Avg. Points per Position,Expected Points,Points Value
2,Max Verstappen,25.4,28.8,Red Bull,29.9,28.775,0.999132
3,Oscar Piastri,29.6,26.6,McLaren,25.4,26.450,0.994361
8,Carlos Sainz,3.5,6.3,Williams,7.1,6.200,0.984127
5,George Russell,22.6,23.0,Mercedes,22.6,22.600,0.982609
0,Lando Norris,29.9,30.8,McLaren,29.6,29.675,0.963474
14,Isack Hadjar,4.3,7.1,Racing Bulls,7.3,6.550,0.922535
10,Fernando Alonso,2.9,6.7,Aston Martin,6.5,5.600,0.835821
7,Liam Lawson,1.9,6.5,Racing Bulls,5.5,4.600,0.707692
17,Oliver Bearman,6.7,7.1,Haas,4.3,4.900,0.690141
1,Charles Leclerc,14.2,22.9,Ferrari,15.8,15.400,0.672489


In [23]:
constructors['Points Value'] = constructors['Expected Points']/constructors['Cost']
constructors.sort_values('Points Value',ascending=False)

,Team,Cost,Expected Points,Points Value
0,McLaren,35.4,56.125,1.585452
2,Red Bull,29.6,35.175,1.188345
3,Mercedes,26.9,31.500,1.171004
1,Ferrari,30.9,30.000,0.970874
7,Racing Bulls,13.6,11.150,0.819853
4,Williams,17.9,14.000,0.782123
6,Aston Martin,12.1,8.800,0.727273
9,Kick Sauber,10.1,7.250,0.717822
8,Haas,13.0,8.400,0.646154
5,Alpine,7.9,1.600,0.202532
